# 🎯 第二章 练习 1: 倾向得分匹配 (Propensity Score Matching)

---

## 从「找相似的人」开始

在第一章，我们知道了随机实验之所以有效，是因为处理组和控制组的特征分布相同。但在观测数据中，这个条件通常不满足。

那怎么办呢？

一个直觉的想法：**既然两组人不一样，那就找一样的呗！**

### 相亲的艺术 💑

想象你是一个相亲平台的算法工程师。你要帮助评估「送玫瑰花」对「相亲成功率」的影响。

问题是：送花的人和不送花的人本来就不一样——
- 送花的人可能更浪漫
- 送花的人可能收入更高（买得起花）
- 送花的人可能更重视这段关系

所以简单比较送花者和不送花者的成功率是有偏的。

**解决方案**: 为每个送花的人，找一个「几乎一模一样但没送花」的人做对比！

这就是**匹配 (Matching)** 的核心思想。

---

## 📚 本节学习目标

1. 理解倾向得分的概念和数学原理
2. 实现倾向得分匹配 (PSM) 算法
3. 评估匹配质量（SMD、方差比）
4. 理解 PSM 的假设和局限性

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from typing import Tuple, List, Optional

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✅ 环境准备完毕！让我们开始学习 PSM！")

---

## 🌟 Part 1: 什么是倾向得分？

### 核心定义

**倾向得分 (Propensity Score)** 是给定协变量 X，个体接受处理的概率：

$$e(X) = P(T=1 | X)$$

### 直觉理解

倾向得分告诉我们：「这个人有多大可能接受处理」

| 场景 | 处理 T | 倾向得分的含义 |
|-----|--------|---------------|
| 营销 | 发优惠券 | 用户多大概率会收到券 |
| 医学 | 服用新药 | 患者多大概率会被处方新药 |
| 教育 | 参加培训 | 员工多大概率会参加培训 |

### 为什么倾向得分如此神奇？

**Rosenbaum & Rubin (1983)** 证明了一个惊人的定理：

> 如果给定 X，潜在结果与 T 独立，那么给定 e(X)，潜在结果也与 T 独立！

$$Y(0), Y(1) \perp T | X \Rightarrow Y(0), Y(1) \perp T | e(X)$$

**翻译成人话**：我们不需要在每个协变量上都匹配，只需要在倾向得分这一个数上匹配就够了！

这大大降低了匹配的难度，因为我们把高维的 X 压缩成了一维的 e(X)。

### 🔬 动手估计倾向得分

In [ ]:
def generate_confounded_data(n: int = 2000, seed: int = 42) -> pd.DataFrame:
    """
    生成有混淆的观测数据
    
    DAG: X -> T, X -> Y, T -> Y
    
    场景: 研究「健身」对「健康指数」的影响
    - X1: 年龄（年轻人更爱健身，健康也更好）
    - X2: 收入（高收入更可能有时间健身）
    - X3: 遗传因素（与健身无关）
    
    真实 ATE = 2
    """
    np.random.seed(seed)
    
    # TODO: 生成三个协变量
    X1 = None  # 👈 你的代码: np.random.randn(n)  # 年龄（标准化）
    X2 = None  # 👈 你的代码: np.random.randn(n)  # 收入
    X3 = None  # 👈 你的代码: np.random.randn(n)  # 遗传因素
    
    # TODO: 生成处理 T（是否健身）
    # 年轻人（X1低）和高收入者（X2高）更可能健身
    # logit(e) = -0.5*X1 + 0.8*X2
    propensity_logit = -0.5 * X1 + 0.8 * X2 if X1 is not None else np.zeros(n)
    propensity = 1 / (1 + np.exp(-propensity_logit))
    T = None  # 👈 你的代码: np.random.binomial(1, propensity)
    
    # TODO: 生成结果 Y（健康指数）
    # Y = 50 + 2*T - 1.5*X1 + 0.8*X2 + 0.5*X3 + noise
    # 真实 ATE = 2
    Y = None  # 👈 你的代码
    
    return pd.DataFrame({
        'X1': X1, 'X2': X2, 'X3': X3, 'T': T, 'Y': Y
    })

In [ ]:
def estimate_propensity_score(X: np.ndarray, T: np.ndarray) -> np.ndarray:
    """
    估计倾向得分 e(X) = P(T=1|X)
    
    使用逻辑回归来估计
    
    Args:
        X: 特征矩阵 (n, p)
        T: 处理状态 (n,)
    
    Returns:
        倾向得分数组 (n,)
    """
    # TODO: 使用 LogisticRegression 拟合 T ~ X
    # 提示: 
    # 1. 创建 LogisticRegression(max_iter=1000)
    # 2. 调用 fit(X, T)
    # 3. 使用 predict_proba(X)[:, 1] 获取 P(T=1|X)
    
    # 👈 你的代码
    pass

In [ ]:
# 测试数据生成和倾向得分估计
df = generate_confounded_data(n=2000, seed=42)

if df['X1'] is not None:
    print("📊 数据概览:")
    print(f"   样本量: {len(df)}")
    print(f"   处理组: {df['T'].sum()} ({df['T'].mean()*100:.1f}%)")
    print(f"   控制组: {(1-df['T']).sum()} ({(1-df['T']).mean()*100:.1f}%)")
    
    # 朴素估计
    naive_ate = df[df['T']==1]['Y'].mean() - df[df['T']==0]['Y'].mean()
    print(f"\n   朴素 ATE 估计: {naive_ate:.4f}")
    print(f"   真实 ATE: 2.0")
    print(f"   偏差: {naive_ate - 2.0:+.4f}")
    
    display(df.head())
else:
    print("❌ 请完成 generate_confounded_data 函数！")

In [ ]:
# 估计倾向得分
if df['X1'] is not None:
    X = df[['X1', 'X2', 'X3']].values
    T = df['T'].values
    Y = df['Y'].values
    
    propensity = estimate_propensity_score(X, T)
    
    if propensity is not None:
        df['propensity'] = propensity
        
        print("📈 倾向得分统计:")
        print(f"   范围: [{propensity.min():.4f}, {propensity.max():.4f}]")
        print(f"   处理组平均: {propensity[T==1].mean():.4f}")
        print(f"   控制组平均: {propensity[T==0].mean():.4f}")
        
        # 可视化
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # 图1: 倾向得分分布
        ax1 = axes[0]
        ax1.hist(propensity[T==1], bins=30, alpha=0.5, label='处理组', color='coral', density=True)
        ax1.hist(propensity[T==0], bins=30, alpha=0.5, label='控制组', color='steelblue', density=True)
        ax1.set_xlabel('倾向得分 e(X)', fontsize=12)
        ax1.set_ylabel('密度', fontsize=12)
        ax1.set_title('倾向得分分布', fontsize=14)
        ax1.legend()
        
        # 图2: 共同支撑
        ax2 = axes[1]
        ax2.scatter(propensity[T==0], np.random.uniform(-0.1, 0.1, (T==0).sum()), 
                   alpha=0.3, label='控制组', c='steelblue', s=20)
        ax2.scatter(propensity[T==1], np.random.uniform(0.9, 1.1, (T==1).sum()), 
                   alpha=0.3, label='处理组', c='coral', s=20)
        
        # 重叠区域
        overlap_min = max(propensity[T==0].min(), propensity[T==1].min())
        overlap_max = min(propensity[T==0].max(), propensity[T==1].max())
        ax2.axvspan(overlap_min, overlap_max, alpha=0.2, color='green', label='共同支撑区域')
        
        ax2.set_xlabel('倾向得分 e(X)', fontsize=12)
        ax2.set_yticks([0, 1])
        ax2.set_yticklabels(['控制组', '处理组'])
        ax2.set_title('共同支撑 (Common Support)', fontsize=14)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("❌ 请完成 estimate_propensity_score 函数！")

---

## 🔗 Part 2: 实现倾向得分匹配

### 匹配算法

最常用的匹配方法是**最近邻匹配 (Nearest Neighbor Matching)**：

1. 对于每个处理组个体，找到倾向得分最接近的控制组个体
2. 这些配对就是「可比较」的
3. 用配对后的数据估计 ATE

### 卡尺匹配 (Caliper Matching)

有时候最近的邻居也可能很远，这时候可以设置一个「卡尺」限制最大距离：

$$|e(X_i) - e(X_j)| < \text{caliper}$$

超过卡尺的匹配将被丢弃。

In [ ]:
def propensity_score_matching(
    propensity: np.ndarray,
    treatment: np.ndarray,
    n_neighbors: int = 1,
    caliper: Optional[float] = None
) -> Tuple[np.ndarray, np.ndarray]:
    """
    执行倾向得分匹配
    
    为每个处理组个体找到倾向得分最接近的控制组个体
    
    Args:
        propensity: 倾向得分
        treatment: 处理状态
        n_neighbors: 匹配的邻居数量
        caliper: 卡尺宽度（最大允许的倾向得分差异）
    
    Returns:
        (matched_treated_indices, matched_control_indices)
    """
    # 获取处理组和控制组的索引
    treated_idx = np.where(treatment == 1)[0]
    control_idx = np.where(treatment == 0)[0]
    
    # TODO: 使用 NearestNeighbors 进行匹配
    # 1. 用控制组的倾向得分训练 KNN
    # 2. 为每个处理组个体找最近的控制组个体
    
    # 👈 你的代码
    # knn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
    # knn.fit(propensity[control_idx].reshape(-1, 1))
    # distances, indices = knn.kneighbors(propensity[treated_idx].reshape(-1, 1))
    
    knn = None
    distances = None
    indices = None
    
    if knn is None:
        return np.array([]), np.array([])
    
    # 应用卡尺约束
    matched_treated = []
    matched_control = []
    
    for i, (dist, idx) in enumerate(zip(distances, indices)):
        # TODO: 如果使用卡尺，检查距离是否在范围内
        if caliper is None or dist[0] <= caliper:
            matched_treated.append(treated_idx[i])
            matched_control.append(control_idx[idx[0]])
    
    return np.array(matched_treated), np.array(matched_control)

In [ ]:
def estimate_ate_psm(
    Y: np.ndarray,
    matched_treated_idx: np.ndarray,
    matched_control_idx: np.ndarray
) -> Tuple[float, float]:
    """
    使用 PSM 估计 ATE
    
    ATE = mean(Y_treated) - mean(Y_control)
    
    Returns:
        (ATE估计, 标准误)
    """
    if len(matched_treated_idx) == 0:
        return None, None
    
    # TODO: 计算匹配后的 ATE
    y_treated = Y[matched_treated_idx]
    y_control = Y[matched_control_idx]
    
    ate = None  # 👈 你的代码: y_treated.mean() - y_control.mean()
    
    # 标准误（简化版）
    diff = y_treated - y_control
    se = None  # 👈 你的代码: diff.std() / np.sqrt(len(diff))
    
    return ate, se

In [ ]:
# 测试 PSM
if propensity is not None:
    matched_t, matched_c = propensity_score_matching(propensity, T, caliper=None)
    
    if len(matched_t) > 0:
        print("🔗 PSM 匹配结果:")
        print(f"   匹配对数: {len(matched_t)}")
        print(f"   匹配率: {len(matched_t) / T.sum() * 100:.1f}%")
        
        # 估计 ATE
        psm_ate, psm_se = estimate_ate_psm(Y, matched_t, matched_c)
        
        if psm_ate is not None:
            print(f"\n📊 ATE 估计:")
            print(f"   PSM ATE: {psm_ate:.4f} ± {psm_se:.4f}")
            print(f"   95% CI: [{psm_ate - 1.96*psm_se:.4f}, {psm_ate + 1.96*psm_se:.4f}]")
            print(f"   偏差: {psm_ate - 2.0:+.4f}")
            print(f"\n   对比朴素估计: {naive_ate:.4f} (偏差: {naive_ate - 2.0:+.4f})")
            
            if abs(psm_ate - 2.0) < abs(naive_ate - 2.0):
                print(f"\n   ✅ PSM 减少了 {abs(naive_ate - 2.0) - abs(psm_ate - 2.0):.2f} 的偏差！")
        else:
            print("❌ 请完成 estimate_ate_psm 函数！")
    else:
        print("❌ 请完成 propensity_score_matching 函数！")

---

## 📏 Part 3: 评估匹配质量

匹配之后，我们需要检查：**处理组和控制组现在真的「可比较」了吗？**

### 标准化均值差 (Standardized Mean Difference, SMD)

$$\text{SMD} = \frac{\bar{X}_{\text{treated}} - \bar{X}_{\text{control}}}{\sqrt{(s^2_{\text{treated}} + s^2_{\text{control}})/2}}$$

**经验法则**: |SMD| < 0.1 表示良好的平衡

In [ ]:
def compute_smd(X_treated: np.ndarray, X_control: np.ndarray) -> np.ndarray:
    """
    计算标准化均值差 (SMD)
    
    SMD = (mean_treated - mean_control) / pooled_std
    """
    # TODO: 计算均值差
    mean_treated = X_treated.mean(axis=0)
    mean_control = X_control.mean(axis=0)
    mean_diff = None  # 👈 你的代码
    
    # TODO: 计算合并标准差
    var_treated = X_treated.var(axis=0)
    var_control = X_control.var(axis=0)
    pooled_std = None  # 👈 你的代码: np.sqrt((var_treated + var_control) / 2)
    
    # TODO: 计算 SMD
    smd = None  # 👈 你的代码: mean_diff / pooled_std
    
    return smd

In [ ]:
def evaluate_balance(
    X: np.ndarray,
    treatment: np.ndarray,
    matched_treated_idx: Optional[np.ndarray] = None,
    matched_control_idx: Optional[np.ndarray] = None,
    feature_names: List[str] = None
) -> Tuple[np.ndarray, np.ndarray]:
    """
    评估匹配前后的协变量平衡
    """
    if feature_names is None:
        feature_names = [f'X{i+1}' for i in range(X.shape[1])]
    
    # 匹配前的 SMD
    smd_before = compute_smd(X[treatment == 1], X[treatment == 0])
    
    # 匹配后的 SMD
    if matched_treated_idx is not None and matched_control_idx is not None:
        smd_after = compute_smd(X[matched_treated_idx], X[matched_control_idx])
    else:
        smd_after = None
    
    return smd_before, smd_after

In [ ]:
# 评估匹配质量
if len(matched_t) > 0:
    smd_before, smd_after = evaluate_balance(X, T, matched_t, matched_c)
    
    if smd_before is not None and smd_after is not None:
        feature_names = ['X1 (年龄)', 'X2 (收入)', 'X3 (遗传)']
        
        print("📏 协变量平衡检查:")
        print("=" * 50)
        print(f"{'变量':<15} {'匹配前 SMD':>12} {'匹配后 SMD':>12} {'改善':>8}")
        print("-" * 50)
        
        for i, name in enumerate(feature_names):
            before = smd_before[i]
            after = smd_after[i]
            improvement = abs(before) - abs(after)
            status = '✅' if abs(after) < 0.1 else '⚠️'
            print(f"{name:<15} {before:>12.4f} {after:>12.4f} {improvement:>+8.4f} {status}")
        
        print("-" * 50)
        print(f"{'平均 |SMD|':<15} {np.abs(smd_before).mean():>12.4f} {np.abs(smd_after).mean():>12.4f}")
        
        # 可视化
        fig, ax = plt.subplots(figsize=(10, 6))
        
        y_pos = np.arange(len(feature_names))
        width = 0.35
        
        bars1 = ax.barh(y_pos - width/2, np.abs(smd_before), width, 
                       label='匹配前', color='coral', alpha=0.7)
        bars2 = ax.barh(y_pos + width/2, np.abs(smd_after), width, 
                       label='匹配后', color='steelblue', alpha=0.7)
        
        ax.axvline(0.1, color='green', linestyle='--', linewidth=2, label='|SMD| = 0.1 阈值')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(feature_names)
        ax.set_xlabel('|SMD|', fontsize=12)
        ax.set_title('匹配前后的协变量平衡', fontsize=14)
        ax.legend(loc='lower right')
        ax.invert_yaxis()
        
        plt.tight_layout()
        plt.show()
    else:
        print("❌ 请完成 compute_smd 函数！")

---

## 🔍 Part 4: 共同支撑检查

### 什么是共同支撑 (Common Support)？

共同支撑假设要求：对于任何协变量值 X，都有一定概率接受或不接受处理：

$$0 < P(T=1|X) < 1$$

**直觉理解**：如果某些人 100% 会接受处理（或 100% 不会），那我们就找不到对照组来比较了。

**例子**：如果只有年收入超过 100 万的人才能买得起某种奢侈保健品，那我们无法用这个数据来推断保健品对普通人的效果。

In [ ]:
def check_common_support(
    propensity: np.ndarray,
    treatment: np.ndarray
) -> dict:
    """
    检查共同支撑假设
    """
    prop_treated = propensity[treatment == 1]
    prop_control = propensity[treatment == 0]
    
    # TODO: 计算重叠区间
    overlap_min = None  # 👈 你的代码: max(prop_treated.min(), prop_control.min())
    overlap_max = None  # 👈 你的代码: min(prop_treated.max(), prop_control.max())
    
    # TODO: 计算在重叠区间外的样本比例
    n_outside = 0
    if overlap_min is not None and overlap_max is not None:
        n_outside = ((propensity < overlap_min) | (propensity > overlap_max)).sum()
    outside_pct = n_outside / len(propensity) if overlap_min is not None else 0
    
    return {
        'treated_min': prop_treated.min(),
        'treated_max': prop_treated.max(),
        'control_min': prop_control.min(),
        'control_max': prop_control.max(),
        'overlap_min': overlap_min,
        'overlap_max': overlap_max,
        'outside_overlap_pct': outside_pct * 100
    }

In [ ]:
# 检查共同支撑
if propensity is not None:
    support = check_common_support(propensity, T)
    
    if support['overlap_min'] is not None:
        print("🔍 共同支撑检查:")
        print("=" * 50)
        print(f"处理组倾向得分范围: [{support['treated_min']:.4f}, {support['treated_max']:.4f}]")
        print(f"控制组倾向得分范围: [{support['control_min']:.4f}, {support['control_max']:.4f}]")
        print(f"重叠区间: [{support['overlap_min']:.4f}, {support['overlap_max']:.4f}]")
        print(f"区间外样本比例: {support['outside_overlap_pct']:.2f}%")
        
        if support['outside_overlap_pct'] < 5:
            print("\n✅ 共同支撑良好！")
        else:
            print("\n⚠️ 有较多样本在共同支撑区间外，可能需要修剪样本。")
    else:
        print("❌ 请完成 check_common_support 函数！")

---

## 🎚️ Part 5: 卡尺宽度的选择

卡尺宽度是一个权衡：

| 卡尺宽度 | 匹配率 | 匹配质量 |
|---------|--------|----------|
| 很小 | 低（很多匹配不上）| 高（匹配很精确）|
| 很大/无限 | 高（都能匹配上）| 低（匹配可能很差）|

常用的卡尺宽度是倾向得分标准差的 0.2 倍。

In [ ]:
def compare_caliper_widths(
    propensity: np.ndarray,
    treatment: np.ndarray,
    Y: np.ndarray,
    caliper_widths: List[float] = [0.01, 0.05, 0.1, 0.2, None]
) -> pd.DataFrame:
    """
    比较不同卡尺宽度的效果
    """
    results = []
    
    for caliper in caliper_widths:
        matched_t, matched_c = propensity_score_matching(propensity, treatment, caliper=caliper)
        
        if len(matched_t) == 0:
            continue
        
        # 计算匹配率
        match_rate = len(matched_t) / treatment.sum()
        
        # 计算 ATE
        ate, se = estimate_ate_psm(Y, matched_t, matched_c)
        
        results.append({
            'caliper': caliper if caliper else '无限',
            'matched_pairs': len(matched_t),
            'match_rate': f"{match_rate:.1%}",
            'ate': ate,
            'se': se,
            'bias': ate - 2.0 if ate else None
        })
    
    return pd.DataFrame(results)

In [ ]:
# 比较不同卡尺
if propensity is not None:
    caliper_results = compare_caliper_widths(propensity, T, Y)
    
    if not caliper_results.empty:
        print("🎚️ 不同卡尺宽度的比较:")
        print("=" * 70)
        display(caliper_results)
        
        print("\n💡 观察:")
        print("   - 卡尺越小，匹配越精确，但匹配对数越少")
        print("   - 卡尺过小可能导致估计不稳定（标准误增大）")
        print("   - 推荐使用 0.2 × std(propensity) 作为卡尺")
        print(f"   - 本数据推荐卡尺: {0.2 * propensity.std():.4f}")

---

## 📝 Part 6: 思考题

### 问题 1: 倾向得分的核心思想是什么？为什么在倾向得分上匹配可以平衡协变量？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 2: PSM 估计的是 ATE 还是 ATT？为什么？

**提示:** 想想我们是为谁找匹配对象的...

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 什么是共同支撑假设？为什么它很重要？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: 如果匹配后某些协变量的 SMD 仍然很大，应该怎么办？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 5: PSM 相比线性回归调整有什么优缺点？

**你的答案:**

*（在这里写下你的思考...）*

---

---

## 🎉 总结

### 核心概念

| 概念 | 定义 | 作用 |
|-----|------|------|
| 倾向得分 | $e(X) = P(T=1|X)$ | 把高维 X 压缩成一维 |
| PSM | 在 e(X) 上寻找最近邻 | 创建可比较的配对 |
| SMD | 标准化均值差 | 评估协变量平衡 |
| 共同支撑 | $0 < e(X) < 1$ | 确保可比较性 |

### PSM 的步骤

1. **估计倾向得分**: 用逻辑回归拟合 $T \sim X$
2. **检查共同支撑**: 确保两组的倾向得分有重叠
3. **执行匹配**: 为每个处理个体找最相似的控制个体
4. **评估平衡**: 检查匹配后的 SMD
5. **估计效应**: 用匹配样本计算 ATE

### 下一步

PSM 有一个问题：它丢弃了很多样本（未被匹配的控制组）。下一个练习我们将学习**逆概率加权 (IPW)**——一种利用所有样本的方法！

---

**「找到对的人比较，才能得出对的结论。」**